#### Neural next-step prediction | part 5: `llmsuggest` co-pilot 
Tutorial on neural theorem proving\
Author: Sean Welleck

----------------

#### High-level goal

Finally, we will see how our neural model can act as a helpful "co-pilot" when we are writing proofs. \
We will make an interactive tool that uses our neural next-step model to suggest next-steps in VSCode.

Concretely, we'll create a `llmsuggest` tactic (in essence, a function) that displays generated suggestions in VSCode. `llmsuggest` is a minimal version of the [**llmstep** [Welleck & Saha 2023]](https://github.com/wellecks/llmstep) tactic, aimed at learning and building off of.

Here is a preview of `llmsuggest`:

<img src="images/llmsuggest/llmsuggest.gif" />


On the top, the user entered `llmsuggest`, then suggestions from our next-step prediction model appear in the Lean Infoview. Clicking a suggestion adds it to the proof.

-----------------------

### High-level approach

Implementing `llmsuggest` involves three components:

1. A Lean *tactic* that sends the current state to a Python script.
2. A Python script that sends the current state to a server via a POST request.
3. A Python server that runs our next-step suggestion model on the current state.

The suggestions (3) are sent back to (2), and the tactic (1) displays the result in VSCode.


### Implementing `llmsuggest`

#### 1. Tactic

At a technical level, the proofs we have seen are actually sequences of *tactics*. 
For instance, `intro` is a tactic and `rw [...]` is a tactic. In general, a *tactic* is a Lean program that manipulates a state. 

To build a new tactic, we use *Lean metaprogramming*, which gives us tools to define new syntax, access the proof state, and more. \
`llmsuggest` only requires basic metaprogramming. To learn more, see the [Lean 4 metaprogramming book](https://github.com/leanprover-community/lean4-metaprogramming-book/tree/master).

`llmsuggest` is implemented in `ntp_lean/LLMsuggest.lean`. The main definition specifies the syntax (i.e., `"llmsuggest"`), then defines the tactic. \
You can see below that the tactic gets the main goal (the "tactic state"), pretty-prints it, and converts it to a string.
Then it runs a `runSuggest` function, and passes the output to an `addSuggestions` function:

```lean
-- `llmsuggest` tactic.
syntax "llmsuggest" : tactic
elab_rules : tactic
  | `(tactic | llmsuggest%$tac) =>
    Lean.Elab.Tactic.withMainContext do
      let goal ← Lean.Elab.Tactic.getMainGoal
      let ppgoal ← Lean.Meta.ppGoal goal
      let ppgoalstr := toString ppgoal
      let suggest ← runSuggest #[ppgoalstr]
      addSuggestions tac $ suggest.splitOn "[SUGGESTION]"
```

The `runSuggest` function calls a Python script (step 2 above), and the `addSuggestions` uses a Lean widget to display the results in VSCode. \
We won't look at these in detail, but please see `ntp_lean/LLMsuggest.lean` if you are curious. \
Hopefully with a small amount of effort, you can modify the tactic or make your own in the future.


#### 2. Python script

The `runSuggest` function in the tactic calls a Python script, `ntp_python/llmsuggest/suggest.py`. It passes the current tactic state as a command line argument.\
The script is simple: it sends a POST request containing the current tactic state to a server:

```python
def suggest(tactic_state):
    conn = http.client.HTTPConnection("localhost", 5000)
    headers = {'Content-type': 'application/json'}
    body = json.dumps({"tactic_state": sys.argv[1]})
    conn.request("POST", "/", body, headers)
    response = conn.getresponse()
    data = response.read()
    data_dict = json.loads(data)
    print('[SUGGESTION]'.join(data_dict['suggestions']))
    conn.close()

if __name__ == "__main__":
    suggest(sys.argv[1])
```

After receiving suggestions, it prints the suggestions, and the printed suggestions will be received in the `runSuggest` function.

#### 3. Server
Finally, in `ntp_python/llmsuggest/server.py` we define a web server that handles the POST request, and hosts the language model.
Specifically, the server initializes our language model, and uses the model to
generate suggestions given a tactic state received in a POST request.
```python
model = transformers.GPTNeoXForCausalLM.from_pretrained('wellecks/llmstep-mathlib4-pythia2.8b')

def generate ...

app = Flask(__name__)

@app.route('/', methods=['POST'])
def process_request():
    data = request.get_json()
    tactic_state = data.get('tactic_state')
    prompt = """[GOAL]%s[PROOFSTEP]""" % (tactic_state)
    texts = generate(prompt, args.num_samples)
    response = {"suggestions": texts}
    return jsonify(response)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=args.port)
```

This server is minimal; one can imagine adding several features.

### Running `llmsuggest`

To run `llmsuggest`, first start the server:
```bash
python python/server.py
```

Then open `ntp_lean/LLMsuggest.lean` in VS Code and try out `llmsuggest`. There are some example theorems and proofs at the bottom of the page:

<img src="images/llmsuggest/llmsuggest_examples.png" alt="" width="450" />

-----------------------

### `llmstep`: [L]LM proofstep suggestions in Lean

[`llmstep`](https://github.com/wellecks/llmstep) is an expanded version of the `llm_suggest` tactic: https://github.com/wellecks/llmstep

`llmstep` includes features such as:
1. **Type checking**: suggestions are checked by Lean and marked as completing a proof, valid, or invalid (but still possibly useful).
2. **Prefixed generation**: e.g. `llmstep "exact"` returns suggestions that start with `"exact"`
3. **Fast inference**: fast inference via [PagedAttention](https://vllm.ai/) for near real-time suggestions
4. **Other models**: support for other models, e.g. `llmstep-llama2`

Here's an example of using `llmstep`:

<img src="images/llmsuggest/llmstep_gif.gif" alt="" width="400" />


The first invocation (`llmstep ""`) gives 5 suggestions, with `intro h n` and `intro h` outlined in blue since they type check.

The second invocation (`llmstep "exact"`) gives suggestions that start with `exact`. The first three are outlined in green since they complete the proof.

----------

## Next steps

This concludes part 1 of the tutorial. We have seen how to build a neural next-step suggestion tool from scratch: collecting data, learning a model, measuring performance with proof search and evaluation sets, and deploying the model as an interactive tactic.

In part 2, we will look at a generalization called language cascades, in which a language model implements a "function" that does more than predict the next step. We will see example cascades for drafting informal proofs, sketching the high-level structure of a proof, and refining proofs.